# Step 1: Install Requirements

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%pip install -qr yolov5/requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 12262, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12262 (delta 2), reused 6 (delta 1), pack-reused 12250
Receiving objects: 100% (12262/12262), 11.96 MiB | 32.93 MiB/s, done.
Resolving deltas: 100% (8488/8488), done.
     |████████████████████████████████| 596 kB 5.1 MB/s 
     |████████████████████████████████| 145 kB 5.1 MB/s 
     |████████████████████████████████| 178 kB 47.3 MB/s 
     |████████████████████████████████| 1.1 MB 57.0 MB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 138 kB 75.7 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 require

In [ ]:
#data download and structure building
!wget http://cg.cs.tsinghua.edu.cn/traffic-sign/data_model_code/data.zip
!unzip data.zip
!gdown --id 12vSm0q0KaV-alySrjG_5P6tiSeeA5Bpe # Download tt100k.yaml file
!mkdir TT100K
!mkdir TT100K/data
!mkdir TT100K/data/train
!mkdir TT100K/data/test
!mkdir TT100K/data/val

!mkdir TT100K/data/train/images
!mkdir TT100K/data/test/images
!mkdir TT100K/data/val/images

!mkdir TT100K/data/train/labels
!mkdir TT100K/data/test/labels
!mkdir TT100K/data/val/labels

!mv data/train/* TT100K/data/train/images
!mv data/test/* TT100K/data/test/images
!mv /content/data/annotations.json TT100K/data

#Save in drive if needed
# !cp -r "/content/TT100K" "/content/drive/MyDrive/"


In [ ]:
!cp -r "/content/TT100K/data/annotations.json" "/content/annotations.json"
!cp -r "/content/TT100K/data/train" "/content/yolov5/data/TT100K"
!cp -r "/content/TT100K/data/test" "/content/yolov5/data/TT100K"

In [ ]:
# load if data stored in Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# !mkdir "/content/yolov5/data/TT100K/"
# !cp -r "/content/drive/MyDrive/TT100K/data/annotations.json" "/content/annotations.json"
# !cp -r "/content/drive/MyDrive/TT100K/data/train" "/content/yolov5/data/TT100K"
# !cp -r "/content/drive/MyDrive/TT100K/data/test" "/content/yolov5/data/TT100K"

Mounted at /content/drive


In [ ]:
dirs=["/content/yolov5/data/TT100K/train/labels/","/content/yolov5/data/TT100K/test/labels/","/content/yolov5/data/TT100K/val/labels/","/content/yolov5/data/TT100K/val/images/"]

for dir in dirs:
  if not os.path.exists(dir):
      os.makedirs(dir)
      
for folder in dirs:
  for filename in os.listdir(folder):
      file_path = os.path.join(folder, filename)
      try:
          if os.path.isfile(file_path) or os.path.islink(file_path):
              os.unlink(file_path)
          elif os.path.isdir(file_path):
              shutil.rmtree(file_path)
      except Exception as e:
          print('Failed to delete %s. Reason: %s' % (file_path, e))



# Label Generation

In [ ]:
import numpy as np
import pandas as pd
import json
import random
import shutil
import math

filedir = "/content/annotations.json"
annos = json.loads(open(filedir).read())
train_category_List=[]
for image in annos['imgs'].values():
  if 'train' in image['path']:
    for object in image['objects']:
      train_category_List.append(object['category'])
validation_pct=0.05
df = pd.DataFrame(dict(col1=np.array(train_category_List)))
c=df['col1'].value_counts()
d=pd.DataFrame({'class':c.index, 'count':c.values})
# top45Classess=d.head(45)['class'].values.tolist()

top45Classess_dict=d.head(45).set_index('class').T.to_dict('int')['count']
validationClasses_Dict={}
for k,v in top45Classess_dict.items():
  validationClasses_Dict[k]={}
  validationClasses_Dict[k]['imageIds']=[]

top45Classess=list(top45Classess_dict.keys())
print(top45Classess_dict)
print(top45Classess)

{'pn': 1886, 'pne': 1379, 'i5': 1045, 'p11': 969, 'pl40': 874, 'po': 735, 'pl50': 660, 'io': 580, 'pl80': 578, 'pl60': 526, 'p26': 515, 'i4': 476, 'pl100': 449, 'pl30': 373, 'il60': 338, 'i2': 297, 'pl5': 269, 'w57': 263, 'p5': 258, 'p10': 244, 'pl120': 208, 'il80': 197, 'ip': 190, 'p23': 163, 'pr40': 136, 'ph4.5': 122, 'w59': 122, 'p3': 111, 'w55': 109, 'pm20': 107, 'p12': 106, 'pg': 104, 'pl70': 103, 'pm55': 98, 'pl20': 98, 'il100': 92, 'w13': 90, 'p19': 87, 'p27': 84, 'ph4': 83, 'pm30': 75, 'wo': 73, 'ph5': 73, 'w32': 70, 'p6': 69}
['pn', 'pne', 'i5', 'p11', 'pl40', 'po', 'pl50', 'io', 'pl80', 'pl60', 'p26', 'i4', 'pl100', 'pl30', 'il60', 'i2', 'pl5', 'w57', 'p5', 'p10', 'pl120', 'il80', 'ip', 'p23', 'pr40', 'ph4.5', 'w59', 'p3', 'w55', 'pm20', 'p12', 'pg', 'pl70', 'pm55', 'pl20', 'il100', 'w13', 'p19', 'p27', 'ph4', 'pm30', 'wo', 'ph5', 'w32', 'p6']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.


In [ ]:
random.seed(33)
baseDir="/content/yolov5/data/TT100K/"
def prepareObj(object):
  x_center=(object['bbox']['xmin']+object['bbox']['xmax'])/2
  y_center=(object['bbox']['ymin']+object['bbox']['ymax'])/2
  width=object['bbox']['xmax']-object['bbox']['xmin']
  height=object['bbox']['ymax']-object['bbox']['ymin']
  x_center/=2048
  y_center/=2048
  width/=2048
  height/=2048
  # return str(annos['types'].index(object['category']))+" "+str(x_center)+" "+str(y_center)+" "+str(width)+" "+str(height)+'\n'
  return str(top45Classess.index(object['category']))+" "+str(x_center)+" "+str(y_center)+" "+str(width)+" "+str(height)+'\n'

def addifTrain(cat,filePath):
  if "train/labels/"in filePath:
    if filePath not in validationClasses_Dict[cat]["imageIds"]: 
      validationClasses_Dict[cat]["imageIds"].append(filePath) 

for image in annos['imgs'].values():
  f=None
  # if 'train' in image['path']:
  #   f = open(baseDir+"train/labels/"+str(image['id'])+".txt", "w")
  if 'test' in image['path']:
    f = open(baseDir+"test/labels/"+str(image['id'])+".txt", "w")
  if f:
    for object in image['objects']:
      if object['category'] in top45Classess:
        addifTrain(object['category'],f.name)
        f.write(prepareObj(object))
    f.close()

# for k,v in validationClasses_Dict.items():

#   listtoMove=random.sample(validationClasses_Dict[k]['imageIds'], math.floor(validation_pct*len(list(validationClasses_Dict[k]['imageIds']))))
#   print(k,len(list(validationClasses_Dict[k]['imageIds'])),listtoMove)
#   for f in listtoMove:
#     img=f.replace("labels","images").replace(".txt",".jpg")
#     if os.path.exists(f):
#       print(f)
#       shutil.move(f, f.replace("train","val"))
#     if os.path.exists(img):
#       print(img)
#       shutil.copyfile(img, img.replace("train","val"))



In [ ]:
%cd yolov5


/content/yolov5


In [ ]:
!python train.py --weights last.pt --data /content/tt100K.yaml --epochs 10 --batch-size -1

train: weights=last.pt, cfg=, data=/content/drive/MyDrive/TT100K/tt100K.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=-1, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-251-gc23a441 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv

In [ ]:
!zip -r /content/yolov5/runs.zip /content/yolov5/runs
!zip -r /content/yolov5/runs.zip /content/yolov5/last.pt
!zip -r /content/yolov5/runs.zip /content/yolov5/best.pt
from google.colab import files
import os
from datetime import datetime
newfile="/content/yolov5/runs_"+str(datetime.now().timestamp())+".zip"
os.rename("/content/yolov5/runs.zip",newfile)
shutil.copy(newfile,"/content/drive/MyDrive/TT100K")

  adding: content/yolov5/runs/ (stored 0%)
  adding: content/yolov5/runs/train/ (stored 0%)
  adding: content/yolov5/runs/train/exp/ (stored 0%)
  adding: content/yolov5/runs/train/exp/P_curve.png (deflated 7%)
  adding: content/yolov5/runs/train/exp/results.csv (deflated 80%)
  adding: content/yolov5/runs/train/exp/hyp.yaml (deflated 45%)
  adding: content/yolov5/runs/train/exp/labels.jpg (deflated 24%)
  adding: content/yolov5/runs/train/exp/val_batch2_labels.jpg (deflated 8%)
  adding: content/yolov5/runs/train/exp/F1_curve.png (deflated 8%)
  adding: content/yolov5/runs/train/exp/R_curve.png (deflated 8%)
  adding: content/yolov5/runs/train/exp/train_batch2.jpg (deflated 1%)
  adding: content/yolov5/runs/train/exp/labels_correlogram.jpg (deflated 19%)
  adding: content/yolov5/runs/train/exp/events.out.tfevents.1655109586.11c1997a4370.7451.0 (deflated 19%)
  adding: content/yolov5/runs/train/exp/val_batch2_pred.jpg (deflated 8%)
  adding: content/yolov5/runs/train/exp/val_batch0_pre

'/content/drive/MyDrive/TT100K/runs_1655123840.616361.zip'

In [ ]:
!python train.py --weights last.pt --data /content/tt100K.yaml --epochs 10 --batch-size -1

train: weights=last.pt, cfg=, data=/content/drive/MyDrive/TT100K/tt100K.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=-1, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 12 (delta 5), reused 10 (delta 5), pack-reused 0
Unpacking objects: 100% (12/12), done.
From https://github.com/ultralytics/yolov5
   c23a441..6a67594  master            -> origin/master
 * [new branch]      test/seeds        -> origin/test/seeds
 

In [ ]:
!zip -r /content/yolov5/runs.zip /content/yolov5/runs
!zip -r /content/yolov5/runs.zip /content/yolov5/last.pt
!zip -r /content/yolov5/runs.zip /content/yolov5/best.pt
from google.colab import files
import os
from datetime import datetime
newfile="/content/yolov5/runs_"+str(datetime.now().timestamp())+".zip"
os.rename("/content/yolov5/runs.zip",newfile)
shutil.copy(newfile,"/content/drive/MyDrive/TT100K")

  adding: content/yolov5/runs/ (stored 0%)
  adding: content/yolov5/runs/val/ (stored 0%)
  adding: content/yolov5/runs/val/exp/ (stored 0%)
  adding: content/yolov5/runs/val/exp/PR_curve.png (deflated 7%)
  adding: content/yolov5/runs/val/exp/val_batch1_labels.jpg (deflated 8%)
  adding: content/yolov5/runs/val/exp/val_batch2_pred.jpg (deflated 7%)
  adding: content/yolov5/runs/val/exp/F1_curve.png (deflated 8%)
  adding: content/yolov5/runs/val/exp/confusion_matrix.png (deflated 21%)
  adding: content/yolov5/runs/val/exp/val_batch2_labels.jpg (deflated 7%)
  adding: content/yolov5/runs/val/exp/val_batch1_pred.jpg (deflated 7%)
  adding: content/yolov5/runs/val/exp/val_batch0_labels.jpg (deflated 8%)
  adding: content/yolov5/runs/val/exp/P_curve.png (deflated 8%)
  adding: content/yolov5/runs/val/exp/R_curve.png (deflated 8%)
  adding: content/yolov5/runs/val/exp/val_batch0_pred.jpg (deflated 8%)
	zip warning: name not matched: /content/yolov5/last.pt

zip error: Nothing to do! (try: 

'/content/drive/MyDrive/TT100K/runs_1655197273.848606.zip'

In [ ]:
!python val.py --data /content/tt100K.yaml --weights best.pt --imgsz 640 --task test

val: data=/content/drive/MyDrive/TT100K/tt100K.yaml, weights=['best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-252-g6a67594 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 290 layers, 21030738 parameters, 0 gradients
100% 755k/755k [00:00<00:00, 15.1MB/s]
test: Scanning '/content/yolov5/data/TT100K/test/labels' images and labels...3067 found, 4 missing, 0 empty, 0 corrupt: 100% 3071/3071 [00:05<00:00, 565.64it/s]
test: New cache created: /content/yolov5/data/TT100K/test/labels.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 96/96 [03:49<00:00,  2.39s/it]
                 all       3071       7698      0.546      0.593      0.573    

In [ ]:
!python detect.py --weights best.pt --data /content/tt100K.yaml --source /content/10286.jpg

detect: weights=['best.pt'], source=/content/10286.jpg, data=/content/drive/MyDrive/TT100K/tt100K.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-253-g75bbaa8 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 290 layers, 21030738 parameters, 0 gradients
image 1/1 /content/10286.jpg: 640x640 3 pl100s, 2 il60s, 1 p5, 2 il80s, Done. (0.014s)
Speed: 0.7ms pre-process, 13.7ms inference, 1.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp10


In [ ]:
!python detect.py --weights best.pt --data /content/tt100K.yaml --source /content/2.jpg